In [40]:
# Importing Dependencies
import numpy as np
import gym
from gym.envs.box2d.car_racing import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam
from statistics import mean
from pyglet.window import key
import time
from tqdm import tqdm 
import os
from keras.utils import normalize

In [2]:
# Function for generating data by playing
def start_playing(n_episodes):
    a = np.array( [0.0, 0.0, 0.0] )
    def key_press(k, mod):
        global restart
        if k==0xff0d: restart = True
        if k==key.LEFT:  a[0] = -1.0
        if k==key.RIGHT: a[0] = +1.0
        if k==key.UP:    a[1] = +1.0
        if k==key.DOWN:  a[2] = +0.8   
    def key_release(k, mod):
        if k==key.LEFT  and a[0]==-1.0: a[0] = 0
        if k==key.RIGHT and a[0]==+1.0: a[0] = 0
        if k==key.UP:    a[1] = 0
        if k==key.DOWN:  a[2] = 0
    
    env = CarRacing()
    env.render()
    env.viewer.window.on_key_press = key_press
    env.viewer.window.on_key_release = key_release
    
    for episode in range(n_episodes):
        prev_observation = env.reset()
        if episode==0:
            X = np.array(prev_observation.tolist())
            y = np.array([0.,0.,0.])
        restart = False
        while True:
            action = a
            observation, reward, done, info = env.step(action)
            X = np.vstack((X, prev_observation))
            y = np.vstack((y, action))
            prev_observation=observation
            env.render()
            if done or restart: break
        time.sleep(2)    
    np.save('train_X.npy', X)
    np.save('train_y.npy', y)    
    env.close()
    return X, y

In [3]:
# Playing the to create data
X, y=start_playing(20)

Track generation: 1121..1405 -> 284-tiles track
Track generation: 925..1162 -> 237-tiles track
retry to generate track (normal if there are not many of this messages)
Track generation: 1228..1538 -> 310-tiles track
Track generation: 1132..1419 -> 287-tiles track
Track generation: 1282..1607 -> 325-tiles track
Track generation: 1171..1472 -> 301-tiles track
retry to generate track (normal if there are not many of this messages)
Track generation: 1120..1404 -> 284-tiles track
Track generation: 1030..1291 -> 261-tiles track
Track generation: 1119..1403 -> 284-tiles track
Track generation: 1126..1412 -> 286-tiles track
Track generation: 1413..1770 -> 357-tiles track
Track generation: 1230..1541 -> 311-tiles track
Track generation: 1246..1562 -> 316-tiles track
Track generation: 1128..1414 -> 286-tiles track
Track generation: 1085..1365 -> 280-tiles track
Track generation: 1387..1738 -> 351-tiles track
Track generation: 992..1251 -> 259-tiles track
Track generation: 1029..1296 -> 267-tiles 

In [8]:
# Combining gameplay files
temp = np.load('train_data-549.4999999999591.npy')
tempX = [i[0] for i in temp]
tempy = [i[1] for i in temp]
X = np.array(tempX)
y = np.array(tempy)
for file in tqdm(os.listdir('Data/')):
    temp = np.load('Data/' + file)
    tempX = [i[0] for i in temp]
    tempy = [i[1] for i in temp]
    tempX = np.array(tempX)
    tempy = np.array(tempy)
    X  = np.concatenate([X, tempX], axis=0)
    y  = np.concatenate([y, tempy], axis=0)

np.save('X.npy', X)    

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:05<00:00,  1.60it/s]


In [5]:
# Converting output to proper OneHot array
Y=[]
for i in range(len(y)):  
    if np.array_equal(y[i], np.array([0,1,0])):
        Y.append(np.array([0,0,1,0,0]))
    elif np.array_equal(y[i], np.array([0,0,0.8])):
        Y.append(np.array([0,0,0,1,0]))
    elif np.array_equal(y[i], np.array([1,0,0])):
        Y.append(np.array([0,1,0,0,0]))
    elif np.array_equal(y[i], np.array([-1,0,0])):
        Y.append(np.array([1,0,0,0,0]))
    else:
        Y.append(np.array([0,0,0,0,1]))      
      
np.save('y_modified.npy', Y)    

In [41]:
# Loading the data
X = np.load('train_X.npy')
y = np.load('y_modified.npy')
X = X.reshape(-1,96,96,3)
print(X.shape)
print(y.shape)

(20450, 96, 96, 3)
(20450, 5)


In [23]:
# Variation in generated data
left=np.array([1,0,0,0,0])
right=np.array([0,1,0,0,0])
straight=np.array([0,0,1,0,0])
stop=np.array([0,0,0,1,0])
no_key=np.array([0,0,0,0,1])
l=0
r=0
s=0
b=0
nk=0

for key in y:
    if np.array_equal(key, left):
        l+=1
    elif np.array_equal(key, right):
        r+=1
    elif np.array_equal(key, straight):
        s+=1
    elif np.array_equal(key, stop):
        b+=1
    elif np.array_equal(key, no_key):
        nk+=1    
      
print(l,r,s,b,nk)    

2694 1318 4443 179 11816


In [56]:
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (96,96,3)))
model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(5, activation = "softmax"))

In [57]:
optimizer = Adam(lr=0.0001)
model.compile(optimizer = optimizer, loss = "categorical_crossentropy")

In [58]:
model.fit(X, y, epochs=10, validation_split=0.2, batch_size=600, verbose=1, shuffle=True)

Train on 16360 samples, validate on 4090 samples
Epoch 1/10
16360/16360 [==============================] - ETA: 53s - loss: 2.71 - ETA: 29s - loss: 2.62 - ETA: 21s - loss: 2.62 - ETA: 17s - loss: 2.57 - ETA: 14s - loss: 2.52 - ETA: 13s - loss: 2.52 - ETA: 11s - loss: 2.51 - ETA: 10s - loss: 2.49 - ETA: 9s - loss: 2.4639 - ETA: 8s - loss: 2.445 - ETA: 7s - loss: 2.415 - ETA: 7s - loss: 2.397 - ETA: 6s - loss: 2.383 - ETA: 6s - loss: 2.367 - ETA: 5s - loss: 2.350 - ETA: 4s - loss: 2.344 - ETA: 4s - loss: 2.343 - ETA: 3s - loss: 2.335 - ETA: 3s - loss: 2.331 - ETA: 3s - loss: 2.322 - ETA: 2s - loss: 2.312 - ETA: 2s - loss: 2.301 - ETA: 1s - loss: 2.286 - ETA: 1s - loss: 2.279 - ETA: 0s - loss: 2.267 - ETA: 0s - loss: 2.254 - ETA: 0s - loss: 2.248 - 12s 736us/step - loss: 2.2458 - val_loss: 1.8947
Epoch 2/10
16360/16360 [==============================] - ETA: 8s - loss: 1.943 - ETA: 8s - loss: 1.937 - ETA: 7s - loss: 1.924 - ETA: 7s - loss: 1.913 - ETA: 7s - loss: 1.931 - ETA: 7s - loss: 1

In [ ]:
# Saving the Model
model.save('Models/first_model')

In [ ]:
# Loading the Model
model.load('Models/first-model')

In [ ]:
#scores = []
#choices = []
env = CarRacing()
for each_game in range(3):
    #score = 0
    #game_memory = []
    prev_obs = env.reset()
    for i in range(1000):
        env.render()
        out = model.predict(prev_obs.reshape(-1,96,96,3))[0]
        print(out)
        
        if out[0]<=0.35:
            out[0]=0  
        if out[1]<=0.2:
            out[1]=0
        if out[2]<=0.01 :
            out[2]=0   
        if out[3]<=0.47:
            out[3]=0
        if out[4]<=9:
            out[4]=0    
        
        output = np.argmax(out)
        action = np.zeros(5)
        np.put(action, output, 1)
        #choices.append(action)
        if np.array_equal(action, np.array([0,0,1,0,0])):
            action=np.array([0,1,0])
        elif np.array_equal(action, np.array([0,0,0,1,0])):
            action=np.array([0,0,0.8])
        elif np.array_equal(action, np.array([0,1,0,0,0])):
            action=np.array([1,0,0])
        elif np.array_equal(action, np.array([1,0,0,0,0])):
            action=np.array([-1,0,0])
        else:
            action=np.array([0,0,0,0,0])
        
        new_observation, reward, done, info = env.step(action)
        print(action,"by nn")
        prev_obs = new_observation
        #game_memory.append([new_observation, action])
        #score+=reward
        if done: 
            break
    #scores.append(score)
    #print('Episode {} Score: {}'.format(each_game+1,score))
    env.close()
#print('Average Score:',sum(scores)/len(scores))

Track generation: 1053..1319 -> 266-tiles track
retry to generate track (normal if there are not many of this messages)
Track generation: 1176..1474 -> 298-tiles track
[1.6442436e-04 2.2818191e-05 2.5397397e-03 1.0105422e-06 9.9727196e-01]
[-1  0  0] by nn
[3.8031849e-01 1.8698987e-01 6.9211383e-05 1.6034690e-01 2.7227548e-01]
[-1  0  0] by nn
[3.0068852e-02 3.3695695e-01 1.6635846e-07 6.7396625e-03 6.2623435e-01]
[1 0 0] by nn
[2.8651420e-04 1.1327161e-01 2.7135920e-09 1.6442219e-03 8.8479763e-01]
[-1  0  0] by nn
[2.54116574e-04 1.00061744e-01 1.62610603e-09 2.22735573e-03
 8.97456765e-01]
[-1  0  0] by nn
[3.6044823e-04 4.5984864e-02 8.9258068e-10 2.6963069e-03 9.5095837e-01]
[-1  0  0] by nn
[8.1289635e-04 4.9936555e-02 3.7725627e-08 5.2610245e-03 9.4398952e-01]
[-1  0  0] by nn
[1.0201948e-01 2.6022905e-01 1.0197162e-05 1.6828600e-01 4.6945527e-01]
[1 0 0] by nn
[5.0902562e-03 1.1353251e-01 3.7955126e-06 1.8559307e-02 8.6281419e-01]
[-1  0  0] by nn
[4.1113165e-03 1.8662386e-01 3.

[-1  0  0] by nn
[0.7452909  0.01796904 0.0042201  0.14888923 0.08363084]
[-1  0  0] by nn
[0.7612344  0.01869725 0.00400068 0.15304963 0.06301811]
[-1  0  0] by nn
[0.80352277 0.01793198 0.00367551 0.13712482 0.03774488]
[-1  0  0] by nn
[0.8573612  0.0155085  0.00288463 0.10654648 0.01769911]
[-1  0  0] by nn
[0.9056179  0.01300782 0.00272077 0.06888136 0.00977216]
[-1  0  0] by nn
[0.9402388  0.01007068 0.00145702 0.04332605 0.00490751]
[-1  0  0] by nn
[0.9666936  0.00655877 0.00111908 0.02387036 0.00175826]
[-1  0  0] by nn
[9.8103720e-01 2.8731183e-03 4.4609801e-04 1.5084035e-02 5.5957068e-04]
[-1  0  0] by nn
[9.8790562e-01 1.3012532e-03 2.9170187e-04 1.0219314e-02 2.8218786e-04]
[-1  0  0] by nn
[9.8781002e-01 1.2273355e-03 3.3113893e-04 1.0313535e-02 3.1799887e-04]
[-1  0  0] by nn
[9.84027863e-01 1.62478583e-03 4.89942438e-04 1.30258165e-02
 8.31613492e-04]
[-1  0  0] by nn
[9.8669469e-01 1.5178721e-03 4.6838634e-04 1.0270450e-02 1.0484791e-03]
[-1  0  0] by nn
[9.8604298e-01

[0.0389013  0.68985456 0.01365943 0.12087542 0.13670924]
[1 0 0] by nn
[0.04117236 0.6348243  0.02382585 0.13142909 0.1687484 ]
[1 0 0] by nn
[0.04832905 0.52412605 0.05254585 0.13899897 0.23600005]
[1 0 0] by nn
[0.05365429 0.40098295 0.12995979 0.14549628 0.26990673]
[1 0 0] by nn
[0.06361601 0.26206425 0.22612877 0.14239404 0.30579698]
[1 0 0] by nn
[0.07692902 0.13606086 0.35474718 0.12708221 0.30518073]
[0 1 0] by nn
[0.12796156 0.08623727 0.3373322  0.11881502 0.32965395]
[0 1 0] by nn
[0.29053408 0.05824173 0.20066877 0.09655613 0.35399923]
[0 1 0] by nn
[0.5300423  0.02968962 0.06778339 0.0656932  0.3067915 ]
[-1  0  0] by nn
[0.73319054 0.01684641 0.0190362  0.04653165 0.18439524]
[-1  0  0] by nn
[0.8347407  0.00774704 0.00692507 0.0366597  0.11392753]
[-1  0  0] by nn
[0.8880217  0.00462702 0.0029029  0.03510603 0.06934225]
[-1  0  0] by nn
[0.8938496  0.00421762 0.00394662 0.03786587 0.06012021]
[-1  0  0] by nn
[0.934039   0.00265309 0.00133221 0.02655294 0.03542275]
[-1  

[0.13417777 0.07990909 0.34030828 0.10708246 0.3385224 ]
[0 1 0] by nn
[0.342109   0.05118238 0.18258736 0.08734462 0.33677667]
[0 1 0] by nn
[0.6395664  0.02148408 0.04609067 0.0561836  0.23667523]
[-1  0  0] by nn
[0.8019577  0.01238102 0.01287419 0.04012456 0.13266261]
[-1  0  0] by nn
[0.8548443  0.00782501 0.00632852 0.03521426 0.09578801]
[-1  0  0] by nn
[0.89389527 0.00550437 0.00327748 0.03292203 0.06440091]
[-1  0  0] by nn
[0.89022595 0.00472165 0.0045593  0.0421292  0.05836398]
[-1  0  0] by nn
[0.92285144 0.00311512 0.00165693 0.02960248 0.04277404]
[-1  0  0] by nn
[0.9205348  0.00275653 0.00169789 0.02670681 0.04830399]
[-1  0  0] by nn
[0.9248957  0.00390262 0.00153425 0.0222098  0.04745758]
[-1  0  0] by nn
[0.90907973 0.00504954 0.00242403 0.02492157 0.0585251 ]
[-1  0  0] by nn
[0.8949352  0.00627662 0.00376457 0.02886573 0.06615788]
[-1  0  0] by nn
[0.88847286 0.00952537 0.00607552 0.03361744 0.0623088 ]
[-1  0  0] by nn
[0.86684865 0.01355105 0.01158035 0.04030789

[0 1 0] by nn
[0.05264684 0.20728327 0.27345756 0.12175971 0.3448526 ]
[0 1 0] by nn
[0.03072377 0.35415804 0.2137003  0.09650207 0.30491582]
[1 0 0] by nn
[0.03192725 0.49528885 0.10980244 0.09952978 0.26345173]
[1 0 0] by nn
[0.02995251 0.5749111  0.07580132 0.10669854 0.21263653]
[1 0 0] by nn
[0.03380436 0.634439   0.04453749 0.11067929 0.17653991]
[1 0 0] by nn
[0.02917134 0.5943169  0.06710353 0.12079008 0.18861812]
[1 0 0] by nn
[0.03299725 0.64640397 0.04101891 0.10856068 0.17101914]
[1 0 0] by nn
[0.02834057 0.522897   0.07051419 0.11493229 0.26331592]
[1 0 0] by nn
[0.0344843  0.45721546 0.07821411 0.11374991 0.31633624]
[1 0 0] by nn
[0.03878265 0.31571326 0.12053327 0.10388977 0.42108104]
[1 0 0] by nn
[0.04237955 0.20297468 0.15864904 0.10262981 0.4933669 ]
[1 0 0] by nn
[0.05614707 0.15233608 0.16296606 0.10704174 0.52150905]
[0 1 0] by nn
[0.08309156 0.09015746 0.153811   0.09984136 0.5730986 ]
[0 1 0] by nn
[0.10905007 0.05779829 0.12325215 0.09621588 0.61368364]
[0 1 0

...

[0.3445285  0.05576985 0.02059003 0.317639   0.26147264]
[0 1 0] by nn
[0.41652593 0.04659536 0.01450883 0.3474675  0.17490242]
[-1  0  0] by nn
[0.63657844 0.02814309 0.00512953 0.23580307 0.09434579]
[-1  0  0] by nn
[0.68175584 0.02346501 0.00410716 0.22174145 0.06893063]
[-1  0  0] by nn
[0.73751116 0.02499432 0.00480048 0.17377004 0.058924  ]
[-1  0  0] by nn
[0.7644842  0.02319626 0.00454892 0.1634198  0.04435078]
[-1  0  0] by nn
[0.79561794 0.01942474 0.0045453  0.14559908 0.03481295]
[-1  0  0] by nn
[0.86668295 0.01449732 0.00318268 0.09638847 0.01924863]
[-1  0  0] by nn
[0.888553   0.01447992 0.0034092  0.07826041 0.01529755]
[-1  0  0] by nn
[0.9001325  0.01316461 0.00385841 0.06812647 0.01471804]
[-1  0  0] by nn
[0.92345023 0.0121526  0.0037932  0.04927615 0.01132779]
[-1  0  0] by nn
[0.9298946  0.01317514 0.00448198 0.0434419  0.00900642]
[-1  0  0] by nn
[0.95144486 0.00630959 0.00270221 0.03260785 0.00693544]
[-1  0  0] by nn
[0.93455505 0.00783923 0.00390687 0.04151

[0.04389786 0.23763119 0.27611604 0.15190183 0.29045308]
[0 1 0] by nn
[0.04298206 0.24712558 0.26899317 0.14875023 0.29214895]
[0 1 0] by nn
[0.04300766 0.25814846 0.26223704 0.14860396 0.28800288]
[0 1 0] by nn
[0.04101745 0.2688709  0.27094224 0.14855285 0.2706165 ]
[0 1 0] by nn
[0.04159296 0.27826345 0.27241877 0.14887868 0.25884616]
[1 0 0] by nn
[0.05165308 0.36859578 0.16828689 0.15518543 0.25627878]
[1 0 0] by nn
[0.06161404 0.28076953 0.19344011 0.15113713 0.3130392 ]
[1 0 0] by nn
[0.07320479 0.22877513 0.24569902 0.14800261 0.30431846]
[0 1 0] by nn
[0.09031578 0.14194311 0.3443552  0.13927431 0.28411162]
[0 1 0] by nn
[0.12629002 0.12737589 0.3392396  0.14179818 0.26529637]
[0 1 0] by nn
[0.10903624 0.07573213 0.48082182 0.12275696 0.21165287]
[0 1 0] by nn
[0.12712714 0.07581781 0.4708072  0.13021718 0.1960307 ]
[0 1 0] by nn
[0.16719441 0.06510473 0.43922836 0.13196275 0.19650978]
[0 1 0] by nn
[0.22198908 0.05859832 0.40446058 0.13148075 0.18347123]
[0 1 0] by nn
[0.283

[9.9998271e-01 4.1273640e-07 2.1736954e-10 1.6986163e-05 1.1282681e-22]
[-1  0  0] by nn
[9.9998784e-01 2.1697500e-07 7.5710764e-11 1.1943161e-05 1.3044486e-23]
[-1  0  0] by nn
[9.99992847e-01 1.11344505e-07 2.41548795e-11 7.08487551e-06
 1.43152123e-24]
[-1  0  0] by nn
[9.9999404e-01 8.4300360e-08 8.8744082e-12 5.8087453e-06 6.1053318e-25]
[-1  0  0] by nn
[9.99992967e-01 1.07365494e-07 7.02927083e-12 6.93806305e-06
 5.88661763e-25]
[-1  0  0] by nn
[9.9999332e-01 1.4218726e-07 8.0630684e-12 6.6021958e-06 2.7279950e-24]
[-1  0  0] by nn
[9.9998021e-01 6.2447543e-07 4.1978546e-11 1.9193683e-05 8.9765010e-22]
[-1  0  0] by nn
[9.9997187e-01 1.3223843e-06 3.3091038e-10 2.6877655e-05 1.1985976e-20]
[-1  0  0] by nn
[9.9994767e-01 4.3373975e-06 1.2647555e-09 4.8064470e-05 4.0118928e-17]
[-1  0  0] by nn
[9.9981886e-01 1.4376774e-05 3.7156960e-09 1.6670805e-04 3.3703417e-14]
[-1  0  0] by nn
[9.9956197e-01 3.1318934e-05 2.3383590e-08 4.0667993e-04 4.8655083e-11]
[-1  0  0] by nn
[9.987835

[3.6697529e-04 9.9163693e-01 1.5340903e-07 2.4675119e-03 5.5284700e-03]
[1 0 0] by nn
[5.2119500e-04 9.8915720e-01 3.8829370e-07 4.8620366e-03 5.4591498e-03]
[1 0 0] by nn
[7.3324831e-04 9.8294765e-01 8.4546724e-07 8.1356782e-03 8.1825480e-03]
[1 0 0] by nn
[1.9137359e-03 9.7644436e-01 9.9051169e-07 7.1960120e-03 1.4444983e-02]
[1 0 0] by nn
[3.0595541e-03 9.7080392e-01 1.8813313e-06 8.1201475e-03 1.8014461e-02]
[1 0 0] by nn
[2.5074931e-03 9.7340333e-01 2.4048184e-06 7.8162998e-03 1.6270474e-02]
[1 0 0] by nn
[2.4289305e-03 9.7029489e-01 7.7685072e-06 1.0166155e-02 1.7102247e-02]
[1 0 0] by nn
[3.0531292e-03 9.6147424e-01 4.9568305e-05 1.7717006e-02 1.7705936e-02]
[1 0 0] by nn
[3.2761146e-03 9.4318330e-01 3.3984412e-04 3.1323839e-02 2.1876788e-02]
[1 0 0] by nn
[0.00722855 0.8914196  0.00315716 0.07351442 0.02468014]
[1 0 0] by nn
[0.00744721 0.83610964 0.01406579 0.11627074 0.02610662]
[1 0 0] by nn
[0.00803021 0.7552996  0.05478101 0.15817435 0.02371487]
[1 0 0] by nn
[0.01168185 0

[5.8560451e-03 9.7470349e-01 5.4593349e-04 1.8393317e-02 5.0128903e-04]
[1 0 0] by nn
[4.3558269e-03 9.7887623e-01 2.2687388e-04 1.5790202e-02 7.5092987e-04]
[1 0 0] by nn
[4.7327308e-03 9.6697491e-01 2.9146831e-04 2.5810670e-02 2.1901985e-03]
[1 0 0] by nn
[7.6304469e-03 9.5699579e-01 5.1272725e-04 2.7798897e-02 7.0621730e-03]
[1 0 0] by nn
[0.00703398 0.9497998  0.00121542 0.03012239 0.01182836]
[1 0 0] by nn
[0.02382325 0.8646669  0.00814087 0.07353824 0.0298308 ]
[1 0 0] by nn
[0.02844156 0.79384977 0.03052283 0.09872183 0.04846405]
[1 0 0] by nn
[0.03468559 0.706579   0.06266429 0.10398234 0.09208877]
[1 0 0] by nn
[0.05648088 0.55380964 0.10970686 0.11979787 0.16020475]
[1 0 0] by nn
[0.05762383 0.38821942 0.25938106 0.1130961  0.18167958]
[1 0 0] by nn
[0.06727998 0.19689544 0.45547685 0.10689315 0.17345458]
[0 1 0] by nn
[0.08926301 0.0836753  0.5785997  0.0918247  0.15663731]
[0 1 0] by nn
[0.19767627 0.04034716 0.5321174  0.07909561 0.1507636 ]
[0 1 0] by nn
[0.4448488  0.023